In [19]:
import pygame
import sys
from pygame.locals import *
from random import *
import math

balls = []
group = pygame.sprite.Group()
bg_size = 617, 557

class Glass(pygame.sprite.Sprite):
    def __init__(self, glass_image, mouse_image, bg_size):
        pygame.sprite.Sprite.__init__(self)
        self.glass_image = pygame.image.load(glass_image).convert_alpha()
        self.glass_rect = self.glass_image.get_rect()
        self.glass_rect.left, self.glass_rect.top = (bg_size[0] - self.glass_rect.width) / 2, bg_size[1] - self.glass_rect.height
        
        self.mouse_image = pygame.image.load(mouse_image).convert_alpha()
        self.mouse_rect = self.mouse_image.get_rect()
        self.mouse_rect.left, self.mouse_rect.top = self.glass_rect.left, self.glass_rect.top
        pygame.mouse.set_pos([self.glass_rect.left, self.glass_rect.top])
        pygame.mouse.set_visible(False)
        
class Ball(pygame.sprite.Sprite):
    def __init__(self, grayball_image, greenball_image, position, speed, bg_size, target):
        pygame.sprite.Sprite.__init__(self)
        self.grayball_image = pygame.image.load(grayball_image).convert_alpha()
        self.greenball_image = pygame.image.load(greenball_image).convert_alpha()
        self.rect = self.grayball_image.get_rect()
        self.rect.left, self.rect.top = position
        self.speed = speed
        self.width, self.height = bg_size[0], bg_size[1]
        self.radius = self.rect.width / 2
        self.target = target
        self.control = False
        #self.rect.center[0] = position[0] + bg_size[0] / 2
        #self.rect.center[1] = position[1] + bg_size[1] / 2
        #self.rect.center[0] = (self.rect.left + self.rect.right) / 2
        #self.rect.center[1] = (self.rect.top + self.rect.bottom) / 2
    def check(self, motion):
        if self.target < motion < self.target + 5:
            return True
        else:
            return False
    def move(self):
        self.rect = self.rect.move(self.speed)
        if self.rect.right < 0:
            self.rect.left = self.width
        elif self.rect.left > self.width:
            self.rect.right = 0
        elif self.rect.bottom < 0:
            self.rect.top = self.height
        elif self.rect.top > self.height:
            self.rect.bottom = 0

def collide_check(item, target):
    col_balls = []
    for each in target:
        distance = math.sqrt(math.pow((item.rect.center[0] - each.rect.center[0]), 2) + math.pow((item.rect.center[1] - each.rect.center[1]), 2))
    if distance <= (item.rect.width + each.rect.width) / 2:
        col_balls.append(each)
    return col_balls





def main():
    pygame.init()
    grayball_image = "aball.png"        
    greenball_image = "bball.png"
    bg_image = "do.png"
    running = True
    width, height = 617, 557
    screen = pygame.display.set_mode(bg_size)
    pygame.display.set_caption('Play the ball - Demo')
    background = pygame.image.load(bg_image).convert_alpha()
    
        
    for i in range(5):
        position = randint(0, width - 100), randint(0, height - 100)
        speed = [randint(-10, 10), randint(-10, 10)]

        ball = Ball(grayball_image, greenball_image, position, speed, bg_size, 5 * (i + 1))
        while pygame.sprite.spritecollide(ball, group, False, pygame.sprite.collide_circle):
            ball.rect.left, ball.rect.top = randint(0, width - 100), randint(0, height - 100)
        balls.append(ball)
        group.add(ball)
        
    
    area = Glass("glass.jpg", 'mouse.png',bg_size)    
    motion = 0
    MYTIMER = USEREVENT + 1
    pygame.time.set_timer(MYTIMER, 1000)
    clock = pygame.time.Clock()
        
    while running:
        for event in pygame.event.get():
            if event.type == QUIT:
                pygame.quit()
                sys.exit()
            elif event.type == MOUSEMOTION:
                motion += 1
            elif event.type == MYTIMER:
                if motion:
                    for each in group:
                        if each.check(motion):
                            each.speed = [0, 0]
                            each.control = True
                    motion = 0
            elif event.type == KEYDOWN:
                if event.key == K_w:
                    for each in group:
                        if each.control:
                            each.speed[1] -= 1
                if event.key == K_s:
                    for each in group:
                        if each.control:
                            each.speed[1] += 1
                if event.key == K_a:
                    for each in group:
                        if each.control:
                            each.speed[0] -= 1 
                if event.key == K_d:
                    for each in group:
                        if each.control:
                            each.speed[0] += 1
        screen.blit(background, (0, 0))
        screen.blit(area.glass_image, area.glass_rect)
        area.mouse_rect.left, area.mouse_rect.top = pygame.mouse.get_pos()
        if area.mouse_rect.left < area.glass_rect.left:
            area.mouse_rect.left = area.glass_rect.left
        if area.mouse_rect.left > area.glass_rect.right - area.mouse_rect.width:
            area.mouse_rect.left = area.glass_rect.right - area.mouse_rect.width
        if area.mouse_rect.top < area.glass_rect.top:
            area.mouse_rect.top = area.glass_rect.top
        if area.mouse_rect.top > area.glass_rect.bottom - area.mouse_rect.height:
            area.mouse_rect.top = area.glass_rect.bottom - area.mouse_rect.height
        screen.blit(area.mouse_image, area.mouse_rect)
        
        for each in balls:
            each.move()
            if each.control:
                screen.blit(each.greenball_image, each.rect)
            else:
                screen.blit(each.grayball_image, each.rect)
 
        for each in group:
            group.remove(each)
            if pygame.sprite.spritecollide(each, group, False, pygame.sprite.collide_circle):
                each.speed[0] = - each.speed[0]
                each.speed[1] = - each.speed[1]
            group.add(each)
                
        pygame.display.flip()
        clock.tick(30)
            
if __name__ == "__main__":
    main()
                

SystemExit: 